# Company Analysis with GenAI

In [1]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
fair_supply = Website("https://www.linkedin.com/company/fairsupply-com-au-pty-limited/")
print(fair_supply.get_contents())

Webpage Title:
Fair Supply | LinkedIn
Webpage Contents:
Skip to main content
LinkedIn
Articles
People
Learning
Jobs
Games
Get the app
Join now
Sign in
Fair Supply
Technology, Information and Internet
Sydney, NSW
4,208 followers
Simplify your ESG Risk Management and Compliance
Follow
View all 41 employees
Report this company
About us
Fair Supply is a cloud-based ESG risk management and compliance solution.

We are a purpose driven ESG technology and data business, specialising in identifying supply chain ESG risk. We help companies identify, manage and mitigate their risk and exposure to modern slavery, greenhouse gas (GHG) carbon emissions, across Scope 1, 2 and 3 and biodiversity loss risk.

We simplify ESG risk management and compliance by using business data combined with supply chain mapping technology to reduce time, cost, and effort. 

Our founders are a human rights lawyer and an industrial mathematician. Subject matter expertise and precision are at the core of our technology.


In [5]:
fair_supply.links


['#main-content',
 '/?trk=organization_guest_nav-header-logo',
 'https://www.linkedin.com/pulse/topics/home/?trk=organization_guest_guest_nav_menu_articles',
 'https://www.linkedin.com/pub/dir/+/+?trk=organization_guest_guest_nav_menu_people',
 'https://www.linkedin.com/learning/search?trk=organization_guest_guest_nav_menu_learning',
 'https://www.linkedin.com/jobs/search?trk=organization_guest_guest_nav_menu_jobs',
 'https://www.linkedin.com/games?trk=organization_guest_guest_nav_menu_games',
 'ms-windows-store://pdp/?ProductId=9WZDNCRFJ4Q7&mode=mini&cid=guest_nav_upsell&trk=organization_guest_guest_nav_menu_windows',
 'https://www.linkedin.com/signup?session_redirect=https%3A%2F%2Fwww%2Elinkedin%2Ecom%2Fcompany%2Ffairsupply-com-au-pty-limited&trk=organization_guest_nav-header-join',
 'https://www.linkedin.com/login?session_redirect=https%3A%2F%2Fwww%2Elinkedin%2Ecom%2Fcompany%2Ffairsupply-com-au-pty-limited&fromSignIn=true&trk=organization_guest_nav-header-signin',
 'https://www.link

## Company analysis

Assemble all the details into another prompt to GPT4-o

In [6]:
link_system_prompt = (
    "You are provided with a list of links found on a LinkedIn page. "
    "Decide which of these links are most relevant for a thorough company analysis—"
    "for example, a company profile, job postings, or employee profiles. "
    "Exclude Terms of Service, Privacy policy, or email links. "
    "You should respond in JSON as in this example:"
)
link_system_prompt += """
{
    "links": [
        {"type": "company profile", "url": "https://www.linkedin.com/company/example-company"},
        {"type": "job posting", "url": "https://www.linkedin.com/jobs/view/123456/"},
        {"type": "employee profile", "url": "https://www.linkedin.com/in/john-doe"}
    ]
}
"""



In [7]:
print(link_system_prompt)

You are provided with a list of links found on a LinkedIn page. Decide which of these links are most relevant for a thorough company analysis—for example, a company profile, job postings, or employee profiles. Exclude Terms of Service, Privacy policy, or email links. You should respond in JSON as in this example:
{
    "links": [
        {"type": "company profile", "url": "https://www.linkedin.com/company/example-company"},
        {"type": "job posting", "url": "https://www.linkedin.com/jobs/view/123456/"},
        {"type": "employee profile", "url": "https://www.linkedin.com/in/john-doe"}
    ]
}



In [8]:
def get_links_user_prompt(website):
    user_prompt = (
        f"Here is the list of links on the website of {website.url}. "
        "Please decide which of these are relevant for a thorough company analysis. "
        "Specifically, we are looking for pages or resources that help assess the company's background, mission, "
        "values, leadership, culture, finances, or job opportunities. "
        "Provide your response in JSON format, listing the full https URLs. "
        "Do not include Terms of Service, Privacy policy, or email links.\n"
        "Links (some might be relative links):\n"
    )
    user_prompt += "\n".join(website.links)
    return user_prompt


In [9]:
print(get_links_user_prompt(fair_supply))

Here is the list of links on the website of https://www.linkedin.com/company/fairsupply-com-au-pty-limited/. Please decide which of these are relevant for a thorough company analysis. Specifically, we are looking for pages or resources that help assess the company's background, mission, values, leadership, culture, finances, or job opportunities. Provide your response in JSON format, listing the full https URLs. Do not include Terms of Service, Privacy policy, or email links.
Links (some might be relative links):
#main-content
/?trk=organization_guest_nav-header-logo
https://www.linkedin.com/pulse/topics/home/?trk=organization_guest_guest_nav_menu_articles
https://www.linkedin.com/pub/dir/+/+?trk=organization_guest_guest_nav_menu_people
https://www.linkedin.com/learning/search?trk=organization_guest_guest_nav_menu_learning
https://www.linkedin.com/jobs/search?trk=organization_guest_guest_nav_menu_jobs
https://www.linkedin.com/games?trk=organization_guest_guest_nav_menu_games
ms-windows

In [10]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [11]:
# Checking links from LinkedIn on Fair Supply LinkedIn page
get_links("https://www.linkedin.com/company/fairsupply-com-au-pty-limited/")

{'links': [{'type': 'company profile',
   'url': 'https://au.linkedin.com/company/fairsupply-com-au-pty-limited'},
  {'type': 'job posting',
   'url': 'https://www.linkedin.com/jobs/search?trk=organization_guest_guest_nav_menu_jobs'},
  {'type': 'employee profile', 'url': 'https://au.linkedin.com/in/cwkwong'},
  {'type': 'employee profile', 'url': 'https://au.linkedin.com/in/jvullinghs'},
  {'type': 'employee profile', 'url': 'https://au.linkedin.com/in/benderson'},
  {'type': 'employee profile',
   'url': 'https://au.linkedin.com/in/denvinlwong'},
  {'type': 'post',
   'url': 'https://www.linkedin.com/posts/fairsupply-com-au-pty-limited_spotlight-due-diligence-by-fair-supply-activity-7239437495967350784-o6eN'},
  {'type': 'post',
   'url': 'https://www.linkedin.com/posts/fairsupply-com-au-pty-limited_google-for-esg-fair-supply-raises-12m-activity-7285533652698963971-Dt6y'},
  {'type': 'post',
   'url': 'https://www.linkedin.com/posts/fairsupply-com-au-pty-limited_australias-mandatory-

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://www.linkedin.com/company/fairsupply-com-au-pty-limited/people/"))

Found links: {'links': []}
Landing page:
Webpage Title:

            
          LinkedIn Login, Sign in | LinkedIn
      
        
Webpage Contents:
Sign in
Sign in with Apple
Sign in with a passkey
or
Email or phone
Password
Show
Forgot password?
Keep me logged in
Sign in
We’ve emailed a one-time link to your primary email address
Click on the link to sign in instantly to your LinkedIn account.
If you don’t see the email in your inbox, check your spam folder.
Resend email
Back
New to LinkedIn?
Join now
Agree & Join LinkedIn
By clicking Continue, you agree to LinkedIn’s
User Agreement
,
Privacy Policy
, and
Cookie Policy
.
LinkedIn
© 2025
User Agreement
Privacy Policy
Community Guidelines
Cookie Policy
Copyright Policy
Send Feedback
Language
العربية (Arabic)
বাংলা (Bangla)
Čeština (Czech)
Dansk (Danish)
Deutsch (German)
Ελληνικά (Greek)
English (English)
Español (Spanish)
فارسی (Persian)
Suomi (Finnish)
Français (French)
हिंदी (Hindi)
Magyar (Hungarian)
Bahasa Indonesia (Indonesian)
It

In [14]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website, \
focusing on a thorough company analysis. Provide a concise overview of the company’s background, mission, \
values, leadership, workplace culture, and job opportunities. Respond in Markdown format. \
Include any details that would be helpful for a prospective candidate evaluating the company, \
such as career growth, employee development, or insights into day-to-day life, if available."


In [15]:
def get_company_analysis_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += (
        "Here are the contents of its linkedin page and other relevant pages. "
        "Use this information to create a concise overview of the company that focuses on a thorough company analysis. "
        "Highlight the company's mission, values, leadership, culture, and job opportunities—"
        "any insights relevant for someone evaluating this company as a potential employer. "
        "Respond in Markdown.\n"
    )
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000]  # Truncate if more than 20,000 characters
    return user_prompt


In [50]:
get_company_analysis_user_prompt("Fair Supply", "https://www.linkedin.com/company/fairsupply-com-au-pty-limited/")

Found links: {'links': [{'type': 'employee profile', 'url': 'https://au.linkedin.com/in/cwkwong?trk=org-employees'}, {'type': 'employee profile', 'url': 'https://au.linkedin.com/in/jvullinghs?trk=org-employees'}, {'type': 'employee profile', 'url': 'https://au.linkedin.com/in/benderson?trk=org-employees'}, {'type': 'employee profile', 'url': 'https://au.linkedin.com/in/denvinlwong?trk=org-employees'}, {'type': 'employee profile', 'url': 'https://au.linkedin.com/in/kimberly-randle-99564370?trk=organization_guest_main-feed-card_reshare-text'}, {'type': 'employee profile', 'url': 'https://au.linkedin.com/in/mark-whittaker-b8791675?trk=organization_guest_main-feed-card_reshare-text'}, {'type': 'employee profile', 'url': 'https://au.linkedin.com/in/arne-geschke-8bb61287?trk=organization_guest_main-feed-card-text'}, {'type': 'company profile', 'url': 'https://au.linkedin.com/company/fairsupply-com-au-pty-limited?trk=organization_guest_main-feed-card_feed-actor-name'}, {'type': 'company profi

'You are looking at a company called: Fair Supply\nHere are the contents of its linkedin page and other relevant pages. Use this information to create a concise overview of the company that focuses on a thorough company analysis. Highlight the company\'s mission, values, leadership, culture, and job opportunities—any insights relevant for someone evaluating this company as a potential employer. Respond in Markdown.\nLanding page:\nWebpage Title:\nFair Supply | LinkedIn\nWebpage Contents:\nSkip to main content\nLinkedIn\nArticles\nPeople\nLearning\nJobs\nGames\nGet the app\nJoin now\nSign in\nFair Supply\nTechnology, Information and Internet\nSydney, NSW\n4,208 followers\nSimplify your ESG Risk Management and Compliance\nFollow\nDiscover all 41 employees\nReport this company\nAbout us\nFair Supply is a cloud-based ESG risk management and compliance solution.\n\nWe are a purpose driven ESG technology and data business, specialising in identifying supply chain ESG risk. We help companies 

In [17]:
def create_company_analysis(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_company_analysis_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [58]:
create_company_analysis("Nebula.io", "https://www.linkedin.com/company/nebula-io/")

Found links: {'links': [{'type': 'employee profile', 'url': 'https://www.linkedin.com/in/eddonner?trk=org-employees'}, {'type': 'employee profile', 'url': 'https://www.linkedin.com/in/jonkrohn?trk=org-employees'}, {'type': 'employee profile', 'url': 'https://www.linkedin.com/in/tomparadis?trk=org-employees'}, {'type': 'employee profile', 'url': 'https://www.linkedin.com/in/andrew-vlahutin?trk=org-employees'}, {'type': 'job posting', 'url': 'https://www.linkedin.com/jobs/scientist-jobs?trk=organization_guest-browse_jobs'}, {'type': 'job posting', 'url': 'https://www.linkedin.com/jobs/director-human-resources-operations-jobs?trk=organization_guest-browse_jobs'}, {'type': 'job posting', 'url': 'https://www.linkedin.com/jobs/director-of-business-development-jobs?trk=organization_guest-browse_jobs'}, {'type': 'job posting', 'url': 'https://www.linkedin.com/jobs/benefits-consultant-jobs?trk=organization_guest-browse_jobs'}, {'type': 'job posting', 'url': 'https://www.linkedin.com/jobs/presid

# Company Overview: Nebula.io

## Background
**Nebula.io**, founded in 2022, is a staffing and recruiting firm headquartered in New York, New York. The company operates as a privately held business, specializing in innovative talent management solutions. With a team of approximately 51-200 employees, Nebula aims to redefine recruitment through its advanced AI-driven platform that connects organizations with a vast talent pool.

## Mission
Nebula.io's mission is to revolutionize the recruitment process by leveraging cutting-edge technology to eliminate traditional hiring challenges. The company seeks to transform how organizations approach talent acquisition, making it more efficient and unbiased.

## Values
1. **Innovation**: Embracing new technologies to improve recruitment outcomes.
2. **Diversity and Inclusion**: Committed to reducing hiring biases with their patented ByeBias™ solution.
3. **Empowerment**: Enabling teams to make better hiring decisions through intuitive tools and robust support.

## Leadership
The leadership team at Nebula.io consists of experienced professionals from the tech and AI sectors. Key figures include:
- **Ed Donner** - Co-Founder & CTO, with a track record of starting AI-driven companies.
- **Jon Krohn** - Co-Founder & Chief Data Scientist, known for his contributions to machine learning and data science education.
- **Tom Paradis** - Head of Product, a product leader specializing in SaaS and AI technologies.

## Workplace Culture
Nebula.io fosters a collaborative and inclusive workplace culture that invites innovation and creativity. Employees are encouraged to share ideas and work together to solve complex problems in recruitment. The company values transparency and promotes a culture of continuous feedback, which helps to grow individual capabilities and strengthen team dynamics.

## Job Opportunities
As Nebula continues to expand its operations, the company frequently seeks talented professionals in various positions ranging from engineering to data science, product management, and more. Job listings can be found directly on their [LinkedIn page](https://signup.nebula.io) and often highlight opportunities for remote and flexible work arrangements.

### Insights for Prospective Candidates
- **Career Growth**: Nebula places a strong emphasis on professional development and skill enhancement. Employees can expect opportunities for advancement and additional training in AI and data science.
- **Employee Development**: The company actively encourages participation in tech conferences and offers resources for continual learning and performance improvement.
- **Day-to-Day Life**: Employees report a dynamic and engaging work environment with a focus on balancing innovation and collaboration. Regular brainstorming sessions and team-building activities help strengthen relationships and foster creativity.

Overall, Nebula.io stands as an attractive option for candidates looking to join a forward-thinking company that is reshaping the future of recruitment through artificial intelligence and smart technology solutions.